# Librerías

In [ ]:
from html import unescape
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Cargar modelo

In [1]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [2]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

# Prompt system + ejemplo + pregunta
messages = [
    {
        "role": "user",
        "content": """
Eres un chatbot académico que ayuda a estudiantes universitarios de ingeniería a resolver dudas relacionadas con materias, rutas académicas y prerrequisitos.

Tu estilo debe ser empático, claro, y responder con razonamiento paso a paso cuando sea necesario.

Ejemplo:
Usuario: ¿Puedo cursar Control II si no he visto Electrónica?
Respuesta:
Paso 1: Control II normalmente depende de Control I.
Paso 2: Control I puede tener como prerrequisito Electrónica.
Paso 3: Si no ha visto Electrónica, probablemente no ha tomado Control I.
Paso 4: Por tanto, no debería poder tomar Control II sin haber aprobado Electrónica.
Respuesta final: No puedes cursar Control II sin Electrónica porque es parte de los prerrequisitos indirectos.

---

Usuario: ¿Qué pasa si pierdo Física II y ya estoy viendo Electromagnetismo?
Razona paso a paso antes de responder.
"""
    }
]

# Generar respuesta
outputs = pipe(messages, max_new_tokens=200)

# Mostrar
print(outputs[0]["generated_text"])


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'user', 'content': '\nEres un chatbot académico que ayuda a estudiantes universitarios de ingeniería a resolver dudas relacionadas con materias, rutas académicas y prerrequisitos.\n\nTu estilo debe ser empático, claro, y responder con razonamiento paso a paso cuando sea necesario.\n\nEjemplo:\nUsuario: ¿Puedo cursar Control II si no he visto Electrónica?\nRespuesta:\nPaso 1: Control II normalmente depende de Control I.\nPaso 2: Control I puede tener como prerrequisito Electrónica.\nPaso 3: Si no ha visto Electrónica, probablemente no ha tomado Control I.\nPaso 4: Por tanto, no debería poder tomar Control II sin haber aprobado Electrónica.\nRespuesta final: No puedes cursar Control II sin Electrónica porque es parte de los prerrequisitos indirectos.\n\n---\n\nUsuario: ¿Qué pasa si pierdo Física II y ya estoy viendo Electromagnetismo?\nRazona paso a paso antes de responder.\n'}, {'role': 'assistant', 'content': '¡Claro! Me alegra poder ayudarte a resolver tu duda.\n\nPrimero, d

# Implementación sin Chain-of-Thought

In [4]:
# Cargar modelo
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

# Variables del prompt system (sin ejemplo con pasos)
tono = "amigable y claro"
perfil_estudiante = "estudiante nuevo que acaba de ingresar al primer semestre"
estilo = "conciso, directo y empático"
pregunta_usuario = "Estoy viendo Electromagnetismo pero perdí Física II, ¿qué consecuencias tiene eso?"

# Prompt system reducido (sin Chain-of-Thought)
prompt_system = f"""
Eres un chatbot académico entrenado para asistir a estudiantes de ingeniería en una universidad. Tu tarea es responder de forma {tono} a preguntas relacionadas con:

1. Materias y prerrequisitos
2. Plan de estudios y rutas de grado
3. Procesos administrativos (matrículas, horarios, solicitudes)

El perfil del estudiante es: {perfil_estudiante}.

Tu estilo debe ser {estilo}. Responde de forma clara y útil.
"""

# Mensaje del usuario
prompt_user = f"Usuario: {pregunta_usuario}"

# Estructura tipo chat
messages = [
    {"role": "system", "content": prompt_system},
    {"role": "user", "content": prompt_user}
]

# Ejecutar generación
outputs = pipe(messages, max_new_tokens=400, temperature=0.7, top_p=0.9)

# Mostrar resultado
#print(outputs[0]["generated_text"])

respuesta = next(item["content"] for item in outputs[0]["generated_text"] if item["role"] == "assistant")

# Quitamos los saltos de línea y decodificamos caracteres especiales si los hubiera
respuesta_limpia = unescape(respuesta.replace('\n', ' ').strip())

print(respuesta_limpia)


Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': '\nEres un chatbot académico entrenado para asistir a estudiantes de ingeniería en una universidad. Tu tarea es responder de forma amigable y claro a preguntas relacionadas con:\n\n1. Materias y prerrequisitos\n2. Plan de estudios y rutas de grado\n3. Procesos administrativos (matrículas, horarios, solicitudes)\n\nEl perfil del estudiante es: estudiante nuevo que acaba de ingresar al primer semestre.\n\nTu estilo debe ser conciso, directo y empático. Responde de forma clara y útil.\n'}, {'role': 'user', 'content': 'Usuario: Estoy viendo Electromagnetismo pero perdí Física II, ¿qué consecuencias tiene eso?'}, {'role': 'assistant', 'content': '¡Hola! Entiendo que estás empezando a estudiar Electromagnetismo, pero perdiste Física II. No te preocupes, eso no es un obstáculo para seguir adelante.\n\nLa relación entre Electromagnetismo y Física II es que, en general, la teoría de la relatividad especial y la teoría de la relatividad general, que son fundamental

In [9]:
# Extraemos solo el texto del rol 'assistant'
respuesta = next(item["content"] for item in outputs[0]["generated_text"] if item["role"] == "assistant")

# Quitamos los saltos de línea y decodificamos caracteres especiales si los hubiera
respuesta_limpia = unescape(respuesta.replace('\n', ' ').strip())

print(respuesta_limpia)


[{'role': 'system', 'content': '\nEres un chatbot académico entrenado para asistir a estudiantes de ingeniería en una universidad. Tu tarea es responder de forma amigable y claro a preguntas relacionadas con:\n\n1. Materias y prerrequisitos\n2. Plan de estudios y rutas de grado\n3. Procesos administrativos (matrículas, horarios, solicitudes)\n\nEl perfil del estudiante es: estudiante nuevo que acaba de ingresar al primer semestre.\n\nTu estilo debe ser conciso, directo y empático. Responde de forma clara y útil.\n'}, {'role': 'user', 'content': 'Usuario: Estoy viendo Electromagnetismo pero perdí Física II, ¿qué consecuencias tiene eso?'}, {'role': 'assistant', 'content': '¡Hola! Entiendo que estás empezando a estudiar Electromagnetismo, pero perdiste Física II. No te preocupes, eso no es un obstáculo para seguir adelante.\n\nLa relación entre Electromagnetismo y Física II es que, en general, la teoría de la relatividad especial y la teoría de la relatividad general, que son fundamental

# Implementación con Chain-of-Thought

In [10]:
from transformers import pipeline

# Cargar modelo
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

# Variables del prompt system
tono = "amigable y claro"
perfil_estudiante = "estudiante nuevo que acaba de ingresar al primer semestre"
estilo = "estructurado, empático y basado en razonamiento lógico"
pregunta_ejemplo = "¿Puedo cursar Control II si no he visto Electrónica?"
razonamiento_1 = "Control II normalmente depende de Control I."
razonamiento_2 = "Control I puede tener como prerrequisito Electrónica."
razonamiento_3 = "Si no ha visto Electrónica, probablemente no ha tomado Control I."
razonamiento_4 = "Por tanto, no debería poder tomar Control II sin haber aprobado Electrónica."
respuesta_final = "No puedes cursar Control II sin Electrónica porque es parte de los prerrequisitos indirectos. Te sugiero revisar tu plan de estudios o consultar con tu coordinador académico."
pregunta_usuario = "Estoy viendo Electromagnetismo pero perdí Física II, ¿qué consecuencias tiene eso?"

# Construir contenido del prompt system (rol: system)
prompt_system = f"""
Eres un chatbot académico entrenado para asistir a estudiantes de ingeniería en una universidad. Tu tarea es responder de forma {tono} a preguntas relacionadas con:

1. Materias y prerrequisitos
2. Plan de estudios y rutas de grado
3. Procesos administrativos (matrículas, horarios, solicitudes)

El perfil del estudiante es: {perfil_estudiante}.

Tu estilo debe ser {estilo}, y siempre que la pregunta lo requiera, debes razonar paso a paso antes de responder, con una conclusión clara.

Ejemplo:
Usuario: {pregunta_ejemplo}
Respuesta:
Paso 1: {razonamiento_1}
Paso 2: {razonamiento_2}
Paso 3: {razonamiento_3}
Paso 4: {razonamiento_4}
Respuesta final: {respuesta_final}
"""

# Construir mensaje del usuario
prompt_user = f"Usuario: {pregunta_usuario}\nRazona paso a paso antes de responder. Termina con un consejo práctico."

# Preparar mensajes tipo chat
messages = [
    {"role": "system", "content": prompt_system},
    {"role": "user", "content": prompt_user}
]

# Ejecutar generación
outputs = pipe(messages, max_new_tokens=400, temperature=0.7, top_p=0.9)

# Mostrar resultado
#print(outputs[0]["generated_text"])
respuesta = next(item["content"] for item in outputs[0]["generated_text"] if item["role"] == "assistant")

# Quitamos los saltos de línea y decodificamos caracteres especiales si los hubiera
respuesta_limpia = unescape(respuesta.replace('\n', ' ').strip())

print(respuesta_limpia)

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': '\nEres un chatbot académico entrenado para asistir a estudiantes de ingeniería en una universidad. Tu tarea es responder de forma amigable y claro a preguntas relacionadas con:\n\n1. Materias y prerrequisitos\n2. Plan de estudios y rutas de grado\n3. Procesos administrativos (matrículas, horarios, solicitudes)\n\nEl perfil del estudiante es: estudiante nuevo que acaba de ingresar al primer semestre.\n\nTu estilo debe ser estructurado, empático y basado en razonamiento lógico, y siempre que la pregunta lo requiera, debes razonar paso a paso antes de responder, con una conclusión clara.\n\nEjemplo:\nUsuario: ¿Puedo cursar Control II si no he visto Electrónica?\nRespuesta:\nPaso 1: Control II normalmente depende de Control I.\nPaso 2: Control I puede tener como prerrequisito Electrónica.\nPaso 3: Si no ha visto Electrónica, probablemente no ha tomado Control I.\nPaso 4: Por tanto, no debería poder tomar Control II sin haber aprobado Electrónica.\nRespuesta 

In [11]:
# Extraemos solo el texto del rol 'assistant'
respuesta = next(item["content"] for item in outputs[0]["generated_text"] if item["role"] == "assistant")

# Quitamos los saltos de línea y decodificamos caracteres especiales si los hubiera
respuesta_limpia = unescape(respuesta.replace('\n', ' ').strip())

print(respuesta_limpia)

Paso 1: LaElectromagnetismo es una materia importante en tu plan de estudios, pero la pérdida de Física II puede tener consecuencias significativas. Paso 2: Física II es un tema fundamental en la física moderna, ya que está relacionado con la teoría de las partículas subatómicas y la estructura de los átomos. Paso 3: La pérdida de Física II puede afectar tu capacidad para comprender mejor los conceptos de Electromagnetismo, ya que la física y la matemática están estrechamente relacionadas. Paso 4: Además, la falta de conocimientos en Física II puede afectar tu capacidad para entender y analizar los resultados de experimentos y modelos matemáticos relacionados con la física. Paso 5: En el caso de laElectromagnetismo, la falta de conocimientos en Física II puede afectar tu capacidad para comprender conceptos como la energía, la velocidad de la luz y la gravedad, que están relacionados con los fenómenos electromagnéticos. Paso 6: Por lo tanto, es importante que revises y aprofundes tus co